An Intro to Python Type Hinting, with Focus on VSCode
=====================================================

A psuedo-history
----------------
In the beginning was Java, and it was statically typed, and it was a PITA. So Python came and rejected static typing, and the type of variable could change from invocation to invocation, and it was good-ish. But as Python applications grew, developers started to miss the certainty which Java's static typing provided, so type hints were born.



In [ ]:
#
# These are type hints.
#
def example(var1: int, var2: int) -> int:
    return var1 * var2

my_number: int = 42

What type hints do
------------------
- provide clues (hence 'hints') to the variable types expected by a function
- allow your IDE to highlight instances where you're passing a type which doesn't match what is expected

What type hints don't do
------------------------
- perform validation[^1]
- change how your code behaves at run time

[^1]: though they do provide the framework for the Pydantic validation library, which DOES do validation

## Enabling type checking in VSCode
To access typechecking in VSCode, install the PyLance extension (default with Python extension) and use menu (Code-> Settings -> Settings) or Cmd+, to open VSCode settings. Then select the desired option under the Extensions > PyLance > Type Checking

Options:
- off - no highlighting of issues
- basic - if you've hinted and done them wrong, it will highlight it
- standard - doesn't seem to be any different from basic? :shrug:
- strict - all the above, plus really finicky highlighting of issues where you haven't hinted things

## Using type hinting in your code

In [1]:
def unhinted(var1, var2):
    return var1 * var2

unhinted(3, 3)

9

In [2]:
unhinted(3, 'bar')

'barbarbar'

### Functions

To type hint parameters to a function (or pretty much anything else) add a `: [type]` immediately following the name of the parameter.

In [3]:
def hinted(var1: int, var2: int):
    return var1 * var2

hinted(3, 3)

9

In [4]:
#
# Notice how the behavior of this method doesn't change, but `bar` is now highlighted in the editor as incorrect.
#
hinted(3, 'bar')

'barbarbar'

#### Type hinting parameters with default values

When type hinting parameters with default values, the pattern is to put the type between the varname and the default declation, like: `{varname}: {type} = {default_value}`

In [5]:
def hinted_default(var1: int, var2: int = 3):
    return var1 * var2

hinted_default(4, 6)

24

In [6]:
hinted_default(4)

12

#### Return type hints 

To type hint the value returned by a function, add `-> [return_type]` after the closing paren in the `def` line.

In [7]:
#
# When type checking is shown, this will flag an issue with `foo`, because it's not an int.
#
def badreturn_hinted(var1: int, var2: int) -> int:
    return 'foo'

#
# Gotcha! This won't flag an issue because booleans are compatible with ints.
#
def gotcha(var1: int, var2: int) -> int:
    return False

assert False == 0

### Hinting complex types

Complex types like lists, dicts, tuples, or sets can use the basic hinting or follow a more detailed notation.

In [8]:
#
# You can specify just the main type and not its content types.
#
def take_generic_list(mylist: list) -> None:
    pass

take_generic_list([1, 2, 3, 4.5, 'bar'])

#
# More robust hinting would also declare the content expected within the base types.
#
def take_list(mylist: list[int]) -> None:
    """
    This expects a variable like:
    [1,2,3]
    """
    pass

# This is highlighted because 4.5 and 'bar' are not int compatible.
take_list([1, 2, 3, 4.5, 'bar'])

def take_dict(mydict: dict[int, str]) -> None:
    """
    This expects a variable like:
    {
        1: 'Foo',
        2: 'Bar'
    }
    """
    pass

take_dict({1: 'foo', 2: 'bar', 3: 42})

#
# Nested objects are specified just like others.
#
def take_nested_dict(nested_dict: dict[int, dict[str, str]]) -> None:
    """
    This will expect something like:
    {
        1: {
            'name': 'Lancelot',
            'favorite_color': 'Blue'
        }
    }
    """
    pass

grail_knights = {
        42: {
            'name': 'Galahad',
            'favorite_color': 'Blue. No yel-- Auuuuuuuugh!'
        }
    }
take_nested_dict(grail_knights)

### Hinting to allow multiple types

The pipe operator (`|`) allows you declare a parameter as handling multiple types.

In [9]:
def multi_types(var1: int | float, var2: int | float) -> None:
    pass

multi_types(3, 9)

multi_types(3, 1.5)

### Hinting classes

Class names can also be used in type hints, and will follow inheritance. So specifying a parent class as the hint will work for child classes as well.

In [10]:
class Foo():
    pass

class Bar(Foo):
    pass

foo = Foo()
bar = Bar()

def handle_foo(my_foo: Foo) -> bool:
    return True

# hinting works with classes, even through inheritance
handle_foo(foo)
handle_foo(bar)

True

#### Using the Collections Abstract Base Class package

The [collections.abc package](https://docs.python.org/3/library/collections.abc.html#collections-abstract-base-classes) provides a bunch of pre-defined types from which other types in Python derive, allowing you to use the abstract base class as a type hint and cover all the types inheriting from it.

In [11]:
from collections.abc import Callable

def invoke_function(func: Callable):
    func()
    
def say_hello():
    print("Hello")

def say_hey():
    print("Hey")

invoke_function(say_hello)
invoke_function(say_hey)
invoke_function("ooops")


Hello
Hey


TypeError: 'str' object is not callable

### Type aliases

Type aliases allow you to define an entity which can be reused in multiple places. So looking at the `grail_knights` dict we defined before, if we needed to pass that structure to several functions instead of just the `take_nested_dict` function, we could define the type alias and then reference that alias in each function.


In [ ]:
#
# This sucks to have to type and eventually I will typo the complex type hint.
#
def take_nested_dict(nested_dict: dict[int, dict[str, str]]):
    """
    This will expect something like:
    {
        1: {
            'name': 'Lancelot',
            'favorite_color': 'Blue'
        }
    }
    """
    pass

def function2(nested_dict: dict[int, dict[str, str]]):
    pass

def function3(nested_dict: dict[int, dict[str, str]]):
    pass

grail_knights = {
        42: {
            'name': 'Galahad',
            'favorite_color': 'Blue. No yel-- Auuuuuuuugh!'
        }
    }



In [ ]:
#
# This is at least a little better.
#
from typing import TypeAlias

GrailKnight:TypeAlias = dict[int, dict[str, str]]

def take_nested_dict2(nested_dict: GrailKnight):
    """
    This will expect something like:
    {
        1: {
            'name': 'Lancelot',
            'favorite_color': 'Blue'
        }
    }
    """
    pass

def function22(nested_dict: GrailKnight):
    pass

def function32(nested_dict: GrailKnight):
    pass

take_nested_dict(grail_knights)
function2(grail_knights)
function3(grail_knights)

In Python 3.12+ we don't need to use the TypeAlias and can instead just use
> type GrailKnight =  dict[int, dict[str, str]]

**Remember: Type hints don't enforce anything**

This is still a little sloppy. It indicated the expected structure of the interior dict, but not actually the _schema_, and it doesn't _enforce_ anything. So, if I'm going to count on each GrailKnight having attributes for `name` and `favorite_color`, I should really just define a GrailKnight class.

#### A better use case?
A better use case for type aliases may be where you want to allow multiple types, like this...

In [ ]:
def first_element(myvar: list | str | tuple) -> int | str | float:
    return myvar[0]

Slicable: TypeAlias = list | str | tuple
Slicable_return: TypeAlias = int | str | float

def first_element2(myvar: Slicable) -> Slicable_return:
    return myvar[0]

### The Any type

There could eventually be a time in which the variable you're defining will accept any variable type that someone wants to throw at it. This _should_ be very, very rare, but if it happens, the Any type is the type hint for you.

You should regard as a form of code smell and make sure it's appropriate and not just someone being lazy, but it's there if you have a need for it.

In [ ]:
from typing import Any

def my_indiscriminate_function(input_var: Any) -> Any:
    print(f'You handed me a variable of type {type(input_var)}. Here it is back.')
    return input_var

my_indiscriminate_function(9999)
my_indiscriminate_function('foo')
my_indiscriminate_function(gotcha)

## Type checking outside the IDE: the mypy package

The [mypy package](https://mypy.readthedocs.io/en/stable/index.html) is installable with pip and lets you do type checking of a file from the command line. This means you could use it as part of a CI workflow or in a githook to ensure your code is type correct.

> $ mypy program.py


When should you use type hints
==============================
- anytime you're developing a library, package, or application which will be used by other people
- ALWAYS, because they:
  - help guide your thinking as you develop
  - show professionalism in your code